In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [26]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("neerajaabhyankar/hindustani-raag-small",split='train[:50%]')
dataset  = pd.DataFrame(dataset)

dataset.head()

Resolving data files:   0%|          | 0/1161 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/92 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1161 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/92 [00:00<?, ?it/s]

,audio,label
0,{'path': '/root/.cache/huggingface/datasets/do...,0
1,{'path': '/root/.cache/huggingface/datasets/do...,0
2,{'path': '/root/.cache/huggingface/datasets/do...,0
3,{'path': '/root/.cache/huggingface/datasets/do...,0
4,{'path': '/root/.cache/huggingface/datasets/do...,0


In [6]:
print(len(dataset))
print(dataset['audio'][0])

580
{'path': '/root/.cache/huggingface/datasets/downloads/4f4c70044eda2ff9cdd894f6ddd15632bfac9f75b3b51e0ec6ce585ba0fae12e', 'array': array([-0.15322396, -0.22031023, -0.18279063, ..., -0.13875762,
       -0.17152269, -0.13755786]), 'sampling_rate': 48000}


In [28]:
from tqdm import tqdm

temp = []
for i in tqdm(range(len(dataset))):
  temp.append(dataset['audio'][i]['path'])

dataset['path'] = temp

100%|██████████| 580/580 [00:00<00:00, 65849.99it/s]


In [29]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   audio   580 non-null    object
 1   label   580 non-null    int64 
 2   path    580 non-null    object
dtypes: int64(1), object(2)
memory usage: 13.7+ KB


In [30]:
print(type(dataset['path'][0]))

<class 'str'>


In [31]:
dataset['path'][dataset['path'].duplicated()]
dataset['path'].isnull().sum()

0

In [32]:
pip install hmmlearn

In [33]:
import os
import librosa
import numpy as np
from sklearn.preprocessing import StandardScaler
from hmmlearn import hmm
from joblib import dump

def extract_segment_mfcc(segment, sr):
    mfccs = librosa.feature.mfcc(y=segment, sr=sr, n_mfcc=5)
    return mfccs.T

def normalize_mfccs(mfccs_array):
    scaler = StandardScaler()
    normalized_mfccs = [scaler.fit_transform(mfccs) for mfccs in mfccs_array]
    return normalized_mfccs

def build_hmm_model(mfcc_features):
    model = hmm.GaussianHMM(n_components=5, covariance_type="diag", n_iter=100)
    model.fit(mfcc_features)
    return model

def calculate_similarity(hmm_models, mfccs_list):
    n_songs = len(hmm_models)
    similarities = np.zeros((n_songs, n_songs))
    likelihoods = []

    for hmm_model in hmm_models:
        hmm_likelihoods = []
        for mfccs in mfccs_list:
            log_likelihood = hmm_model.score(mfccs)
            hmm_likelihoods.append(log_likelihood)
        likelihoods.append(hmm_likelihoods)

    for i in range(n_songs):
        for j in range(n_songs):
            Ni = len(likelihoods[i])
            Nj = len(likelihoods[j])
            sim = ((likelihoods[i][i]-likelihoods[i][j]))/Ni+((likelihoods[j][j]-likelihoods[j][i]))/Nj
            if sim==0:
                similarities[i, j] = sim
                continue
            similarities[i, j] = np.log(sim)

    return similarities

hmm_models = []
mfccs_list = []
file_names=[]
cnt=0
for filename in dataset['path']:
    file_names.append(filename)
    y, sr = librosa.load(filename)
    segment_length = 2000
    n_samples_per_segment = int((segment_length / 1000) * sr)
    n_segments = len(y) // n_samples_per_segment

    mfccs = [extract_segment_mfcc(y[i * n_samples_per_segment:(i + 1) * n_samples_per_segment], sr)
                    for i in range(n_segments)]

    concatenated_mfccs = np.concatenate(mfccs)
    mfccs_list.append(concatenated_mfccs)
    cnt+=1

normalized_mfccs = normalize_mfccs(mfccs_list)
print(file_names)
for m in range(len( mfccs_list)):
    hmm_model = build_hmm_model( mfccs_list[m])
    hmm_models.append(hmm_model)
    dump(hmm_model, f'{file_names[m]}.pkl')

['/root/.cache/huggingface/datasets/downloads/4f4c70044eda2ff9cdd894f6ddd15632bfac9f75b3b51e0ec6ce585ba0fae12e', '/root/.cache/huggingface/datasets/downloads/1208e38b23fc5dc1fcd57a3fdcb98e7376a86b7a2ee492c6816715d7da04410b', '/root/.cache/huggingface/datasets/downloads/eadb40fc2c05ecdccc0f6ef6c8bc45958a8605154ffe5d7eeda87cec93722c0a', '/root/.cache/huggingface/datasets/downloads/5a77fbb813627ffd29f22dad9b73f993260ff181a1b209d5e1fd074f6cedcca2', '/root/.cache/huggingface/datasets/downloads/ac1fc1ba258add3712c27d49c2886b3a330ee6751af0923e03540763ec90c931', '/root/.cache/huggingface/datasets/downloads/244e2e58276689d693b9d276dd415d4a3f819d5069c9d52e74f01834e71b484b', '/root/.cache/huggingface/datasets/downloads/42cc0fe6be084275dcb56b9c674acf9a7e6e5b0a9e8f10bf9e8845b9c94e9eeb', '/root/.cache/huggingface/datasets/downloads/4e62430e1f31e75eecb39221a70801852b8a7845d7ff79269cca1c10931e9ad6', '/root/.cache/huggingface/datasets/downloads/78648294619853f08114a3b2231520d4fd3f1266b7e4695c81ee042110

In [34]:
from joblib import dump

cnt=1
for i, hmm_model in enumerate(hmm_models):
    dump(hmm_model, f'{cnt}.pkl')
    cnt+=1

In [35]:
from joblib import load

hmm_models = []
for filename in dataset['path']:
    hmm_model = load(f'{filename}.pkl')
    hmm_models.append(hmm_model)

In [36]:
print(len(hmm_models))
print(hmm_models)

580
[GaussianHMM(n_components=5, n_iter=100), GaussianHMM(n_components=5, n_iter=100), GaussianHMM(n_components=5, n_iter=100), GaussianHMM(n_components=5, n_iter=100), GaussianHMM(n_components=5, n_iter=100), GaussianHMM(n_components=5, n_iter=100), GaussianHMM(n_components=5, n_iter=100), GaussianHMM(n_components=5, n_iter=100), GaussianHMM(n_components=5, n_iter=100), GaussianHMM(n_components=5, n_iter=100), GaussianHMM(n_components=5, n_iter=100), GaussianHMM(n_components=5, n_iter=100), GaussianHMM(n_components=5, n_iter=100), GaussianHMM(n_components=5, n_iter=100), GaussianHMM(n_components=5, n_iter=100), GaussianHMM(n_components=5, n_iter=100), GaussianHMM(n_components=5, n_iter=100), GaussianHMM(n_components=5, n_iter=100), GaussianHMM(n_components=5, n_iter=100), GaussianHMM(n_components=5, n_iter=100), GaussianHMM(n_components=5, n_iter=100), GaussianHMM(n_components=5, n_iter=100), GaussianHMM(n_components=5, n_iter=100), GaussianHMM(n_components=5, n_iter=100), GaussianHMM

In [59]:
print(filename)

/root/.cache/huggingface/datasets/downloads/0e6c73f4f296b38832c0ee1d89191ca420061173ece79473d89f5577a456b89f


In [63]:
# Add a function to classify the pahar of the day
def classify_pahar(hour):

    if hour is None:
        return "Prahar 0"
    if 6 <= hour < 12:
        return "Prahar 1"
    elif 12 <= hour < 17:
        return "Prahar 2"
    elif 17 <= hour < 20:
        return "Prahar 3"
    elif 20 <= hour < 24:
        return "Prahar 4"
    elif 0 <= hour < 4:
        return "Prahar 5"
    else:
        return "Prahar 6"

In [80]:
def calculate_similarity(hmm_models, mfccs_list, hour):
    pahar = classify_pahar(hour)
    n_songs = len(hmm_models)
    similarities = np.zeros((n_songs, n_songs))
    likelihoods = []

    # Calculate log likelihoods for each HMM model
    for i, hmm_model in tqdm(enumerate(hmm_models)):
        hmm_likelihoods = []
        for mfccs in mfccs_list:
            log_likelihood = hmm_model.score(mfccs)  # Calculate the log likelihood
            hmm_likelihoods.append(log_likelihood)
        likelihoods.append(hmm_likelihoods)

        Nj=0

        # Calculate similarity between HMM models
        for j in (range(n_songs)):
            if i == j:
                similarities[i, j] = 0
            elif i<len(likelihoods) and j<len(likelihoods):
                Ni = len(likelihoods[i])
                Nj = len(likelihoods[j])
                #print(likelihoods)
                if i < Ni and j < Nj and j < Nj:
                    sim = ((likelihoods[i][i]-likelihoods[j][i]))/Ni+((likelihoods[j][i]-likelihoods[j][j]))/Nj
                    if sim==0:
                        similarities[i, j] = sim
                    else:
                        similarities[i, j] = np.log(sim)
                else:
                    similarities[i, j] = 0

    return similarities

hour = 9
# Calculate the similarity between HMM models
similarities = calculate_similarity(hmm_models, mfccs_list, hour)

print(similarities)

# Recommend songs based on the similarity matrix
def recommend_songs(similarities, song_index, n_recommendations):
    recommended_songs = []
    for i in np.argsort(-similarities[song_index]):
        if i!= song_index:
            recommended_songs.append(i)
        if len(recommended_songs) == n_recommendations:
            break
    return recommended_songs


2it [00:01,  1.30it/s]<ipython-input-80-4ceff2bf809e>:30: RuntimeWarning: invalid value encountered in log
  similarities[i, j] = np.log(sim)
5it [00:02,  2.44it/s]<ipython-input-80-4ceff2bf809e>:30: RuntimeWarning: invalid value encountered in log
  similarities[i, j] = np.log(sim)
6it [00:02,  2.60it/s]<ipython-input-80-4ceff2bf809e>:30: RuntimeWarning: invalid value encountered in log
  similarities[i, j] = np.log(sim)
7it [00:03,  2.77it/s]<ipython-input-80-4ceff2bf809e>:30: RuntimeWarning: invalid value encountered in log
  similarities[i, j] = np.log(sim)
8it [00:03,  2.88it/s]<ipython-input-80-4ceff2bf809e>:30: RuntimeWarning: invalid value encountered in log
  similarities[i, j] = np.log(sim)
9it [00:03,  2.95it/s]<ipython-input-80-4ceff2bf809e>:30: RuntimeWarning: invalid value encountered in log
  similarities[i, j] = np.log(sim)
10it [00:04,  2.99it/s]<ipython-input-80-4ceff2bf809e>:30: RuntimeWarning: invalid value encountered in log
  similarities[i, j] = np.log(sim)
11it 

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.01225095 0.         0.         ... 0.         0.         0.        ]
 [       nan        nan 0.         ... 0.         0.         0.        ]
 ...
 [2.38286572 2.2847861  2.54800832 ... 0.         0.         0.        ]
 [2.35559189 2.25465895 2.52493461 ...        nan 0.         0.        ]
 [2.76467985 2.69878278 2.88030276 ... 1.61701832 1.67327412 0.        ]]


In [81]:
song_index = 0
n_recommendations = 5
recommended_songs = recommend_songs(similarities, song_index, n_recommendations)
print("Recommended songs for song", song_index, ":", recommended_songs)

Recommended songs for song 0 : [383, 384, 385, 386, 387]
